# MIMIC-CXR 데이터 전처리 전략

## 목적
MIMIC-CXR 데이터는 방대한 양의 X-ray 이미지와 라벨 정보를 포함하고 있으나, 다음과 같은 이유로 **노이즈 제거 및 샘플 균형 조절**이 필요합니다:

- 동일 환자가 지나치게 많은 X-ray 이미지를 갖는 경우 → 편향 위험
- 다양한 촬영 자세(ViewPosition)로 인한 이미지 변동성
- 보조기구(Support Devices)로 인한 시각적 잡음

---

## 🧪 필터링 조건 요약

| 조건 | 설명 | 목적 |
|------|------|------|
| **촬영 자세(View Position)** | 가장 많이 사용된 자세만 유지 (예: PA 또는 AP) | 자세 불일치로 인한 노이즈 제거 |
| **보조기구 미포함** | `Support Devices`가 `NaN` 또는 `0.0`인 경우만 유지 | 튜브, 선 등의 시각적 요소 제거 |
| **환자당 최대 N개 이미지 유지** | 동일 환자(`subject_id`) 당 최대 5개의 X-ray만 포함 | 데이터 불균형 방지 및 모델 일반화 성능 확보 |

---

## 전처리 후 기대 효과

- 데이터셋 내 **샘플 균형 개선**
- **촬영 조건 표준화**로 모델 훈련 안정성 향상
- 불필요한 시각적 요소 제거로 **진단 관련 특징에 집중 가능**
- 환자 간 분산 확보 → **overfitting 방지**

## 1. ✅ 6개 이상 이미지 가진 환자 제거
- `subject_id` 기준으로 이미지 개수 집계
- 5개 이상 가진 환자는 **전체 제외** (데이터 과대표집 방지)

In [5]:
import pandas as pd

# 1. 데이터 불러오기
chexpert_path = 'mimic-cxr_csv/mimic-cxr-2.0.0-chexpert.csv'
metadata_path = 'mimic-cxr_csv/mimic-cxr-2.0.0-metadata.csv'

df_chexpert = pd.read_csv(chexpert_path)
df_metadata = pd.read_csv(metadata_path)

# 2. subject_id 기준 이미지 수 세고, 6개 이상 가진 환자 제외
image_counts = df_chexpert['subject_id'].value_counts()
subjects_to_exclude = image_counts[image_counts >= 6].index
df_filtered = df_chexpert[~df_chexpert['subject_id'].isin(subjects_to_exclude)]

In [6]:
study_counts_per_patient = df_filtered.groupby("subject_id")["study_id"].nunique()
study_distribution = study_counts_per_patient.value_counts().sort_index()

print(study_distribution)

study_id
1    32697
2    10943
3     5671
4     3572
5     2416
Name: count, dtype: int64


## 2. ✅ Support Devices 없는 환자만 필터링
- `Support Devices == 0.0` 또는 `NaN`만 유지
- 튜브, 카테터 등 **진단과 무관한 시각적 요소 제거**

In [7]:
# 3. Support Devices 없는 경우만 남기기 (0.0 또는 NaN)
df_filtered = df_filtered[
    (df_filtered['Support Devices'] == 0.0) | 
    (df_filtered['Support Devices'].isna())
]

## 3. ✅ 메타데이터 병합
- `mimic-cxr-2.0.0-metadata.csv`와 `subject_id`, `study_id` 기준 병합
- **ViewPosition**, `StudyDate`, `dicom_id` 등 정보 추가 확보

In [8]:
# 4. 메타데이터 병합 (subject_id, study_id 기준)
df_merged = pd.merge(df_filtered, df_metadata, on=["subject_id", "study_id"], how="inner")


## 4. ✅ 가장 흔한 View Position만 유지 (예: PA 또는 AP)
- `ViewPosition` 기준으로 가장 많은 값 하나만 유지
- 자세 불일치로 인한 이미지 다양성 제거

In [9]:
# 5. 가장 많이 등장한 ViewPosition 값 선택
most_common_view = df_merged['ViewPosition'].value_counts().idxmax()

In [10]:
# 6. 해당 ViewPosition만 필터링
df_final = df_merged[df_merged['ViewPosition'] == most_common_view]

## 5.최종 데이터 특징

In [11]:
# 7. 결과 확인
print(f"최종 샘플 수: {len(df_final)}")
print(f"가장 흔한 ViewPosition: {most_common_view}")
print(df_final[['subject_id', 'study_id', 'dicom_id', 'ViewPosition']].head())

최종 샘플 수: 57380
가장 흔한 ViewPosition: PA
    subject_id  study_id                                      dicom_id  \
0     10000032  50414267  02aa804e-bde0afdd-112c0b34-7bc16630-4e384014   
2     10000032  53189527  2a2277a9-b0ded155-c0de8eb9-c124d10e-82c5caab   
11    10000898  50771383  2a280266-c8bae121-54d75383-cac046f4-ca37aa16   
14    10000898  54205396  b75df1bd-0f22d631-52d73526-2ae7b85a-d843b39d   
15    10001038  58224503  28fad2ac-d6001216-b4f72c5b-2d4d452e-17b6c9a5   

   ViewPosition  
0            PA  
2            PA  
11           PA  
14           PA  
15           PA  


In [12]:
# 1. 질병 라벨 컬럼만 선택 (subject_id, study_id, Support Devices 제외)
label_cols = df_chexpert.columns[2:-1]  # CheXpert의 질병 라벨 컬럼 순서 유지

# 2. df_final에 존재하는 질병 컬럼만 필터 (혹시 일부 빠졌을 수도 있음)
available_labels = [col for col in label_cols if col in df_final.columns]

# 3. 각 라벨에서 1.0의 비율 계산 (%)
positive_ratios = (df_final[available_labels] == 1.0).sum() / len(df_final) * 100

# 4. 정렬 및 출력
positive_ratios = positive_ratios.sort_values(ascending=False).round(2)
print(positive_ratios)


No Finding                    63.38
Lung Opacity                  10.75
Atelectasis                    6.90
Cardiomegaly                   6.83
Pleural Effusion               5.58
Pneumonia                      5.25
Lung Lesion                    2.36
Fracture                       1.92
Edema                          1.80
Consolidation                  1.14
Enlarged Cardiomediastinum     1.09
Pleural Other                  0.67
Pneumothorax                   0.65
dtype: float64


In [13]:
positive_counts = (df_final[available_labels] == 1.0).sum().sort_values(ascending=False)


print(positive_counts)

No Finding                    36368
Lung Opacity                   6171
Atelectasis                    3961
Cardiomegaly                   3921
Pleural Effusion               3202
Pneumonia                      3015
Lung Lesion                    1355
Fracture                       1104
Edema                          1033
Consolidation                   657
Enlarged Cardiomediastinum      625
Pleural Other                   383
Pneumothorax                    374
dtype: int64


### 레이블 분포 확인

In [14]:
# 1. 질병 라벨 컬럼 추출
label_cols = df_chexpert.columns[2:-1]
available_labels = [col for col in label_cols if col in df_final.columns]

# 2. 상태별 개수 집계
label_distribution = pd.DataFrame({
    'Positive (1.0)': (df_final[available_labels] == 1.0).sum(),
    'Negative (0.0)': (df_final[available_labels] == 0.0).sum(),
    'Uncertain (-1.0)': (df_final[available_labels] == -1.0).sum(),
    'Missing (NaN)': df_final[available_labels].isna().sum()
})

# 3. 정렬 및 출력
label_distribution = label_distribution.sort_values(by='Positive (1.0)', ascending=False)
print(label_distribution)


                            Positive (1.0)  Negative (0.0)  Uncertain (-1.0)  \
No Finding                           36368               0                 0   
Lung Opacity                          6171             702               606   
Atelectasis                           3961             193              1307   
Cardiomegaly                          3921            3350               414   
Pleural Effusion                      3202            5696               666   
Pneumonia                             3015            7406              3034   
Lung Lesion                           1355             419               341   
Fracture                              1104             407               238   
Edema                                 1033            4974               977   
Consolidation                          657            2186               470   
Enlarged Cardiomediastinum             625            1434               470   
Pleural Other                          3

In [16]:
# 불확실(-1)과 결측(NaN)을 모두 0으로 처리
df_final.loc[:, available_labels] = df_final[available_labels].replace(-1.0, 0.0).fillna(0.0)

### 특정 라벨 데이터만 남기기

In [18]:
# 드랍할 컬럼만 지정
columns_to_drop = [
    "Fracture", "Lung Lesion", "Pleural Other", "Consolidation",
    "Edema", "Enlarged Cardiomediastinum", "Pneumothorax",
    "Support Devices"
]

# 해당 컬럼들만 삭제
df_final = df_final.drop(columns=columns_to_drop)


In [19]:
print(df_final.columns.tolist())


['subject_id', 'study_id', 'Atelectasis', 'Cardiomegaly', 'Lung Opacity', 'No Finding', 'Pleural Effusion', 'Pneumonia', 'dicom_id', 'PerformedProcedureStepDescription', 'ViewPosition', 'Rows', 'Columns', 'StudyDate', 'StudyTime', 'ProcedureCodeSequence_CodeMeaning', 'ViewCodeSequence_CodeMeaning', 'PatientOrientationCodeSequence_CodeMeaning']


# 파일저장

In [20]:
# 파일 경로 설정 (원하는 파일명으로 바꾸세요)
output_path = "mimic-cxr_csv/filtered_mimic_cxr_final.csv"

# CSV 파일로 저장
df_final.to_csv(output_path, index=False)

# 저장 완료 메시지 (선택)
print(f"파일이 저장되었습니다: {output_path}")


파일이 저장되었습니다: mimic-cxr_csv/filtered_mimic_cxr_final.csv
